In [31]:
# Pull Sportsnet Player Lists
import urllib3
import certifi
from bs4 import BeautifulSoup
import pandas as pd

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

base_url = "https://fantasy.sportsnet.ca/sportsnet/baseball18/player_stats"
group_param = "pick_stats_group"

scrape_pages = {
    "C/DH": 1,
    "1B/3B": 2,
    "2B/SS": 3,
    "OF": 4,
    "SP": 5,
    "RP": 6
}

frames = {}

for key, value in scrape_pages.items():

    response = http.request("GET", base_url + "?" + group_param + "=" + str(value))
    
    soup = BeautifulSoup(response.data, 'html.parser')
    table = soup.find('table', attrs={'id': 'playerstats'})
    header = [th.text for th in table.find('thead').select('th')]
    players = [[td.text.strip().split('\n')[0] for td in row.select('td')] for row in table.tbody.find_all('tr')]
    cols = zip(*players)
    tbl_d = {name: col for name, col in zip(header, cols)}
    df = pd.DataFrame(tbl_d, columns=header)
    name = df['Player'].str.split(', ')
    df["First"] = name.str[1]
    df['Last'] = name.str[0]
    frames[key] = df